# DS2.Gen(pp)

## Matrix Generation

In [6]:
import hashlib
from sage.all import *
from sage.stats.distributions.discrete_gaussian_integer import DiscreteGaussianDistributionIntegerSampler
from sage.modules.free_module_element import vector as sage_vector

# Parameters
number = 2 ** 46
next_prime_number = next_prime(number) #generate prime number  > 2 ** 46
q = next_prime_number + 5
n = 256
k, l = 2, 2
sigma = 3  # Standard deviation for Gaussian sampler
party_number = 2

# Define the ring Rq
R.<x> = PolynomialRing(ZZ)
Zq = Integers(q)
Rq = PolynomialRing(Zq, 'x').quotient(x^n + 1)

def discrete_gaussian_sampler(std_dev, ring):
    """
    Sample a polynomial from the discrete Gaussian distribution over a ring.
    """
    D = DiscreteGaussianDistributionIntegerSampler(sigma=std_dev)
    return ring([D() for _ in range(party_number)])

def commit(matrix, party_number):
    """
    Create a commitment of the matrix using the party number.
    """
    combined = str(matrix) + str(party_number)
    return hashlib.sha256(combined.encode()).hexdigest()

# H1 function representing the random oracle commitment
def H1(matrix, party_number):
    return commit(matrix, party_number)

# Generate k×l matrices using discrete Gaussian samplers
matrices = []
for _ in range(party_number):
    matrix = Matrix(Rq, k, l, lambda i, j: discrete_gaussian_sampler(sigma, Rq))
    matrices.append(matrix)
    #print(matrices)

# Commitments for each party
gn = []
for i, matrix in enumerate(matrices):
    commitment = H1(matrix, i + 1)
    gn.append(commitment)

print("Commitments stored in gn:", gn)


Commitments stored in gn: ['437e1c81caf86a6e7b68433ee5037287f8676d1c5df6c958000d195d153b2a09', 'ba9e7bfa60308bc193057f7f19cf657ca2ecfb9ef63ff1a7aec984adccb37491']


In [7]:
# 1. Upon receiving gj for all j ∈ [n − 1] send out An.
An = matrices[-1]
print("Sending out An:", An)

# 2. Upon receiving Aj for all j ∈ [n − 1]:
abort_flag = False
for j, Aj in enumerate(matrices[:-1]):
    if H1(Aj, j+1) != gn[j]:
        abort_flag = True
        break

if abort_flag:
    print("Sending out: abort")
else:
    A = sum(matrices)
    I = identity_matrix(Rq, k)
    A_bar = A.augment(I)
    print("Public random matrix A¯:\n", A_bar)

Sending out An: [70368744177679*xbar + 70368744177681              3*xbar + 70368744177682]
[             2*xbar + 70368744177683              70368744177683*xbar + 3]
Public random matrix A¯:
 [             70368744177681*xbar + 3 70368744177683*xbar + 70368744177681                                    1                                    0]
[             9*xbar + 70368744177680              70368744177683*xbar + 2                                    0                                    1]


## Key Pair Generation

In [8]:
# H2 function representing the random oracle commitment
def H2(matrix, party_number):
    return commit(matrix, party_number)

# Parameters
eta = 5  # Bound for Sη

# 1. Sample a secret key share sn and compute a public key share tn
def discrete_gaussian_sampler_sk(std_dev, ring, bound, n):
    """
    Sample a polynomial from the discrete Gaussian distribution over a ring.
    Ensure that the coefficients are within the given bound.
    """
    D = DiscreteGaussianDistributionIntegerSampler(sigma=std_dev)
    coeffs = []
    for _ in range(party_number):
        val = D()
        while abs(val) > bound:  # Ensure that the value is within the bound
            val = D()
        coeffs.append(val)
    return ring(coeffs)

# Sample sn using the discrete Gaussian distribution
sn = discrete_gaussian_sampler_sk(eta, Rq, eta, n)
tn = A_bar * sn
g_prime_n = H2(tn, party_number)
print("Sending out g'n:", g_prime_n)

# 2. Upon receiving g'j for all j ∈ [n − 1] send out tn.
print("Sending out tn:", tn)

# 3. Upon receiving tj for all j ∈ [n − 1]:
abort_flag = False
t_values = [tn]  # Assuming tn is already received

# Mock storage for g'j values (for the sake of this example)
g_prime_values = [g_prime_n]  # Assuming g_prime_n is already received

for j in range(party_number-1):  # Simulating the reception of other t values
    tj = A_bar * sn
    t_values.append(tj)
    
    # Mock generation and storage of g'j values
    g_prime_j = H2(tj, j+1)
    g_prime_values.append(g_prime_j)
    
    if H2(tj, j+1) != g_prime_values[j+1]:  # Check against stored g'j values
        abort_flag = True
        break

if abort_flag:
    print("Sending out: abort")
else:
    t_combined = sum(t_values)
    print("Combined public key t:", t_combined)

# Local output for Pn
skn = sn
pk = (A, t_combined)
print("Local output for Pn (skn, pk):", skn, pk)

Sending out g'n: 606adb37e0585d3b67df4cae9472cdf38e0a860b2353d61f37ee20cb75bd2c85
Sending out tn: [            70368744177672*xbar^2 + 70368744177681*xbar + 15 70368744177680*xbar^2 + 70368744177667*xbar + 70368744177669                                                   4*xbar + 5                                                            0]
[                        36*xbar^2 + 29*xbar + 70368744177664                          70368744177680*xbar^2 + 3*xbar + 10                                                            0                                                   4*xbar + 5]
Combined public key t: [            70368744177660*xbar^2 + 70368744177678*xbar + 30 70368744177676*xbar^2 + 70368744177650*xbar + 70368744177654                                                  8*xbar + 10                                                            0]
[                        72*xbar^2 + 58*xbar + 70368744177644                          70368744177676*xbar^2 + 6*xbar + 20                   

# Protocol DS3.Signn(sid, skn, pk, µ)

## Inputs

In [9]:
# H3 function for computing the per-message commitment key
def H3(message, public_key):
    combined = str(message) + str(public_key)
    return hashlib.sha256(combined.encode()).hexdigest()

# 1. Pn receives the inputs
sid = "unique_session_id_123"  # Example session ID
used_sids = set()  # Set to keep track of used session IDs
message = "example_message"

# 2. Pn verifies that sid has not been used before
if sid in used_sids:
    print("Session ID has been used before. Protocol not executed.")
else:
    used_sids.add(sid)
    
    # 3. Pn locally computes the per-message commitment key
    ck = H3(message, pk)
    print("Per-message commitment key ck:", ck)

Per-message commitment key ck: 3a28354cfe417df03c17e531b6ff93703da88f787f33a151346a3ac608b173bd


## Signature Generation

In [26]:
# Parameters
eta = 5
k, l = 2, 2
N = 256
alpha = 2  # Example value for alpha
kappa = 60  # Example value for kappa
T = kappa * eta * sqrt(N * (l + k))
sigma = alpha * T
gamma = 1.1  # Example value for gamma
B = gamma * sigma * sqrt(N * (l + k))

# # Step 1: definition of polynomial ring
# N = 256
# r = 5  # Example value for r
# R.<X> = PolynomialRing(ZZ)
# f = X^N + 1
# Rq = R.quotient_ring(f, 'x')

# # Step 2: ノルムの定義
# def norm_infinity(x):
#     return max([abs(coeff) for coeff in x.coefficients()])

# # Step 3: Srの定義
# def Sr(r):
#     return [x for x in Rq if norm_infinity(x) <= r]

# elements_in_Sr = Sr(r)


# H4 function
def H4(commitment):
    commitment_str = str(commitment)
    return hashlib.sha256(commitment_str.encode()).hexdigest()

# H0 function
def H0(com, message, public_key):
    combined = str(com) + str(message) + str(public_key)
    hashed = hashlib.sha256(combined.encode()).digest()
    return vector(ZZ, hashed)[:l+k]  # Convert the hash to a vector in R

# Commit function
def Commit(ck, msg, r=None):
    if r is None:  # If r is not provided, sample it from D(Sr)
        r = ZZ.random_element(-Sr, Sr + 1)
    combined = str(ck) + str(msg) + str(r)
    return hashlib.sha256(combined.encode()).hexdigest()

# Openck function
def Openck(ck, com, r, msg):
    if com == Commit(ck, msg, r):
        return 1
    else:
        return 0

def rejection_sampling(c, s, z):
    print("Starting rejection_sampling...")  # 追加

    # Convert each element of z to a Sage vector if it's a tuple or list
    z_converted = []
    for zi in z:
        if isinstance(zi, (list, tuple, sage.modules.vector_integer_dense.Vector_integer_dense)):
            z_converted.append(vector(ZZ, zi))
        else:
            z_converted.append(zi)  # Keep the integer as it is

    print("z_converted:", z_converted)  # 追加

    # Convert c and s to vectors if they are not already
    if not isinstance(c, sage.modules.vector_integer_dense.Vector_integer_dense):
        c = vector(ZZ, [c])
    if not isinstance(s, sage.modules.vector_integer_dense.Vector_integer_dense):
        s = vector(ZZ, s)

    print("c and s converted to vectors.")  # 追加

    # Compute Ds and Dcs
    Ds_values = []
    Dcs_values = []
    for zi in z_converted:
        if isinstance(zi, sage.modules.vector_integer_dense.Vector_integer_dense):
            Ds_val = exp(-zi.norm()**Integer(2) / (Integer(2) * sigma**Integer(2)).n())**(l+k)  # .n() for numerical evaluation
            Dcs_val = exp(-(zi - vector(ZZ, [c[i]*s[i] for i in range(len(c))])).norm()**Integer(2) / (Integer(2) * sigma**Integer(2)).n())**(l+k)  # .n() for numerical evaluation
            
            # 0の負の指数での累乗を防ぐためのチェック
            if Ds_val != 0:
                Ds_values.append(Ds_val)
            else:
                Ds_values.append(1)  # 0を避けるための適当な値

            if Dcs_val != 0:
                Dcs_values.append(Dcs_val)
            else:
                Dcs_values.append(1)  # 0を避けるための適当な値
        else:
            Ds_values.append(1)  # For integers, just append 1
            Dcs_values.append(1)  # For integers, just append 1

    print("Ds_values and Dcs_values computed.")  # 追加

    Ds = prod(Ds_values)
    Dcs = prod(Dcs_values)

    print("Ds and Dcs computed.")  # 追加

    t = 2

    print("Starting to compute M...")  # 追加
    M = e^t/alpha + 1/(2*alpha^2)
    print("M computed:", M)  # 追加

    print("Starting to compute Ds / (M * Dcs)...")  # 追加
    ratio = Ds / (M * Dcs)
    simplified_ratio = float(ratio)  # 式の簡略化
    print("Simplified Ds / (M * Dcs):", simplified_ratio)  # 簡略化された式を表示

    print("Starting to compute result...")  # 追加
    result = random() < min(1, simplified_ratio)  # 簡略化された式を使用
    print("Result computed:", result)  # 追加


    print("Rejection sampling result:", result)  # 追加

    return result



# Signature Generation
def signature_generation(message, skn, pk):
    print("Starting signature generation...")  # 追加
    
    # 1. Compute the first message
    print("Generating yn...")  # 追加
    
    # Generate k×l matrices using discrete Gaussian samplers
    yns = [vector(Rq, [discrete_gaussian_sampler(sigma, Rq) for _ in range(l+k)]) for _ in range(party_number)]
    
    print("Calculating wn...")  # 追加
    wn = [A_bar * yn for yn in yns]  # Modified to apply for each yn in yns

    print("Calculating ck...")  # 追加
    ck = H3(message, pk)

    print("Generating rn...")  # 追加
    rn = discrete_gaussian_sampler(sigma, Rq)  # Sample r for comn

    print("Calculating comn...")  # 追加
    
    #CSetup
    m = 3
    m2 = 6
   
    #homomorphic commitment    
    #CGen
    Im = identity_matrix(Rq, m)
    Ik = identity_matrix(Rq, k)
    Z = zero_matrix(Rq, k, m)
    A1hatdash = Matrix(Rq, m, m, lambda i, j: discrete_gaussian_sampler(sigma, Rq))
    A2hatdash = Matrix(Rq, k, m-k, lambda i, j: discrete_gaussian_sampler(sigma, Rq))
    A1hat = Im.augment(A1hatdash)
    A2hat = Z.augment(Ik).augment(A2hatdash)

    #Commitck
    r = vector(Rq, [discrete_gaussian_sampler(sigma, Rq) for _ in range(m2)])
    zero_m = vector(ZZ, m, [0]*m)
    wn_converted = []
    for wi in wn:
        if isinstance(wi, (list, tuple, sage.modules.vector_integer_dense.Vector_integer_dense)):
            wn_converted.append(vector(ZZ, wi))
        else:
            wn_converted.append(wi)  # Keep the polynomial as it is
    concatenated_list = list(zero_m) + wn_converted
    concatenated_vector = column_matrix([concatenated_list])
    new_matrix = block_matrix([[A1hat], [A2hat]])
    new_matrix_r = new_matrix * r
    new_matrix_r_list = list(new_matrix_r)
    new_matrix_r = Matrix(5, 1, new_matrix_r_list)
    print(new_matrix_r)
    print(concatenated_vector)
    f1_f2 = new_matrix_r + concatenated_vector

    print("Calculating hn...")  # 追加
    hn = H4(f1_f2)

    
    # Simulating the reception of other h values
    h_values = [hn]  # Assuming hn is already received
    r_values = [rn]  # Store the r values for each commitment
    com_values = [f1_f2]  # Assuming comn is already received

    for j in range(n-1):
        rj = discrete_gaussian_sampler(sigma, Rq)
        r_values.append(rj)

        # Calculate the message part once
        message_part = A_bar * vector([ZZ.random_element(-int(sigma), int(sigma) + 1) for _ in range(l+k)])

        # Use the same message part for both h_values and com_values
        h_values.append(H4(Commit(ck, message_part, rj)))
        com_values.append(Commit(ck, message_part, rj))
    
    print("Starting commitment check...")  # 追加

    abort_flag = False
    for j, comj in enumerate(com_values):
        print(f"Checking commitment at index {j}...")  # 追加
        if H4(comj) != h_values[j]:
            print(f"Mismatch at index {j}:")
            print(f"H4(comj): {H4(comj)}")
            print(f"h_values[j]: {h_values[j]}")
            abort_flag = True
            break

    print("Finished commitment check...")  # 追加
    
    if abort_flag:
        return "abort"
    else:
        com = ''.join(map(str, com_values))
        c = H0(com, message, pk)
        zn = [c * skn] + list(yns)

        if rejection_sampling(c, skn, zn):
            # Receive (zj, rj) for all j in [n-1]
            z_values = [zn]  # Assuming zn is already received
            for j in range(n-1):
                zj = vector([ZZ.random_element(-int(B), int(B) + 1) for _ in range(l+k)])
                z_values.append(zj)

            # a. For each j in [n-1], reconstruct wj and validate the signature share
            for j, zj in enumerate(z_values):
                print(type(A_bar))
                print(type(zj))
                print(type(c))
                print(type(t_values))
                zj = vector(ZZ, zj)
                print(type(zj))
                t_values = vector(ZZ, t_values)
                print(type(t_values))
                wj = A_bar * zj - c * t_values[j]
                if zj.norm(2) > B or Openck(ck, com_values[j], r_values[j], wj) == 0:
                    return "abort"

            # b. Compute z and r
            z = sum(z_values)
            r = sum(r_values)

            # c. If the protocol does not abort, Pn obtains a signature (com, z, r) as local output
            return (com, z, r)
        else:
            return "restart"


        
# Example usage
result = signature_generation(message, skn, pk)
if result == "abort":
    print("Protocol aborted.")
elif result == "restart":
    print("Restarting the protocol.")
else:
    com, z, r = result
    print("Commitment com:", com)
    print("Signature z:", z)
    print("Random value r:", r)

Starting signature generation...
Generating yn...
Calculating wn...
Calculating ck...
Generating rn...
Calculating comn...


TypeError: no common canonical parent for objects with parents: 'Integer Ring' and 'Ambient free module of rank 2 over Univariate Quotient Polynomial Ring in xbar over Ring of integers modulo 70368744177684 with modulus x^256 + 1'

## Verification(incompleted)

In [ ]:
from sage.matrix.constructor import Matrix

# Verification Algorithm
def verification_algorithm(message, signature, pk):
    com, z, r = signature
    A_bar, t_combined = pk
    
    # zをMatrix型に変換
    z_rows = []
    max_length = max(len(item) if isinstance(item, tuple) else 1 for item in z)
    for item in z:
        if isinstance(item, tuple):
            z_rows.append(list(item) + [Integer(0)]*(max_length - len(item)))
        else:
            z_rows.append([item] + [Integer(0)]*(max_length - 1))
    z_matrix = Matrix(z_rows)
    
    # t_combinedがMatrix型でない場合、Matrix型に変換（必要に応じて）
    if not isinstance(t_combined, Matrix):
        t_combined = Matrix(t_combined)
    
    ck = H3(message, pk)
    c = H0(com, message, pk)
    w = A_bar * z_matrix - c * t_combined
    if z_matrix.norm(2) <= Bn and Openck(ck, com, r, w) == 1:
        return True
    else:
        return False

# Example usage
message = "Hello_World"  # Sample message for testing
signature = (com, z, r)  # Sample signature from the provided signature algorithm output

result = verification_algorithm(message, signature, pk)
print(result)
